In [ ]:
from google.colab import files
files.upload()

In [2]:
!mkdir -p ~/.kaggle
!mv "kaggle (1).json" ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d shaunthesheep/microsoft-catsvsdogs-dataset

Dataset URL: https://www.kaggle.com/datasets/shaunthesheep/microsoft-catsvsdogs-dataset
License(s): other
 97% 761M/788M [00:04<00:00, 42.5MB/s]
100% 788M/788M [00:04<00:00, 197MB/s] 


In [14]:
import zipfile
import os
os.makedirs("dataset", exist_ok=True)
with zipfile.ZipFile("microsoft-catsvsdogs-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset")

In [15]:
!ls dataset/PetImages

Cat  Dog


In [16]:
cat_image = os.listdir('/content/dataset/PetImages/Cat')
dog_image = os.listdir('/content/dataset/PetImages/Dog')
print(len(cat_image),len(dog_image))

12501 12501


In [17]:
import os

def remove_undecodable_images_tf(directory):
    print(f"Checking directory for undecodable images using TensorFlow: {directory}")
    for root, _, files in os.walk(directory):
        for file in files:
            filepath = os.path.join(root, file)

            if os.path.isdir(filepath):
                continue
            try:
                img_bytes = tf.io.read_file(filepath)
                img = tf.image.decode_image(img_bytes, channels=3)

                if tf.size(img).numpy() == 0:
                    print(f"Removing empty image: {filepath}")
                    os.remove(filepath)

            except tf.errors.InvalidArgumentError as e:
                print(f"Removing undecodable image: {filepath} - {e}")
                os.remove(filepath)
            except Exception as e:
                print(f"Could not process image {filepath}: {e}")

dataset_dir = "/content/dataset/PetImages"
cat_dir = os.path.join(dataset_dir, "Cat")
dog_dir = os.path.join(dataset_dir, "Dog")

remove_undecodable_images_tf(cat_dir)
remove_undecodable_images_tf(dog_dir)

print("Undecodable image removal complete using TensorFlow.")




Checking directory for undecodable images using TensorFlow: /content/dataset/PetImages/Cat
Removing undecodable image: /content/dataset/PetImages/Cat/666.jpg - {{function_node __wrapped__DecodeImage_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input is empty. [Op:DecodeImage] name: 
Removing undecodable image: /content/dataset/PetImages/Cat/10404.jpg - {{function_node __wrapped__DecodeImage_device_/job:localhost/replica:0/task:0/device:CPU:0}} Unknown image file format. One of JPEG, PNG, GIF, BMP required. [Op:DecodeImage] name: 
Removing undecodable image: /content/dataset/PetImages/Cat/Thumbs.db - {{function_node __wrapped__DecodeImage_device_/job:localhost/replica:0/task:0/device:CPU:0}} Unknown image file format. One of JPEG, PNG, GIF, BMP required. [Op:DecodeImage] name: 
Removing undecodable image: /content/dataset/PetImages/Cat/4351.jpg - {{function_node __wrapped__DecodeImage_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input size should match (header_size + r

In [18]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense,Input,Flatten,Rescaling

In [19]:
train_dir = tf.keras.utils.image_dataset_from_directory(
    directory = '/content/dataset/PetImages',
    labels = 'inferred',label_mode='int',batch_size=32,image_size=(224,224),
    validation_split = 0.2,subset='training',seed=42
)

test_dir = tf.keras.utils.image_dataset_from_directory(
    directory = '/content/dataset/PetImages',
    labels = 'inferred',label_mode='int',batch_size=32,image_size=(224,224),
    validation_split = 0.2,subset='validation',seed=42
)


Found 24991 files belonging to 2 classes.
Using 19993 files for training.
Found 24991 files belonging to 2 classes.
Using 4998 files for validation.


## **VGG16**

In [20]:
model = VGG16(
    include_top = False,
    weights = 'imagenet',input_shape=(224,224,3),classes=2
)

model.trainable = False

input = Input(shape=(224,224,3))
x = model(input)
x = Rescaling(1./255)(x)
x = Flatten()(x)
output = Dense(1,activation='sigmoid')(x)

model = tf.keras.models.Model(inputs = input,outputs=output)
model.compile(optimizer='Adam',loss='BinaryCrossentropy',metrics=['accuracy'])


In [21]:
model.fit(train_dir,batch_size=32,epochs=2,validation_data=test_dir)

Epoch 1/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 156s 246ms/step - accuracy: 0.9417 - loss: 0.2249 - val_accuracy: 0.9734 - val_loss: 0.0818
Epoch 2/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 137s 219ms/step - accuracy: 0.9798 - loss: 0.0727 - val_accuracy: 0.9752 - val_loss: 0.0680


# Finetuning with VGG16

In [23]:
for layer in model.layers:
  print(layer.name,layer.trainable)

input_layer_3 True
vgg16 False
rescaling_1 True
flatten_1 True
dense_1 True


In [34]:
model_vgg = VGG16(
    include_top = False,
    weights = 'imagenet',input_shape=(224,224,3),classes=2
)

model_vgg.trainable = True
set_trainable = False

for layer in model_vgg.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

# for layer in model_vgg.layers:
#   print(layer.name,layer.trainable)

input = Input(shape=(224,224,3))
x = Rescaling(1./255)(input)
x = model_vgg(x)
x = Flatten()(x)
output = Dense(1,activation='sigmoid')(x)
model = tf.keras.models.Model(inputs= input,outputs=output)

model.compile(optimizer='Adam',loss='BinaryCrossentropy',metrics=['accuracy'])

model_vgg.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 7,079,424 (27.01 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

In [35]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_4 (Rescaling)         │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │        25,089 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,739,777 (56.23 MB)

 Trainable params: 7,104,513 (27.10 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

In [33]:
model.fit(train_dir,batch_size=32,epochs=2,validation_data=test_dir)

Epoch 1/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 165s 254ms/step - accuracy: 0.8543 - loss: 0.2966 - val_accuracy: 0.9598 - val_loss: 0.1039
Epoch 2/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 153s 245ms/step - accuracy: 0.9708 - loss: 0.0753 - val_accuracy: 0.9690 - val_loss: 0.0902


# Data_Argomentation

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
train_data = train_datagen.flow_from_directory(
    directory="/content/dataset/PetImages",
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="training",
    seed=42
)

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)
valid_data = valid_datagen.flow_from_directory(
    directory="/content/dataset/PetImages",
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="validation",
    seed=42
)



**Keras Augmentation Layers**

In [ ]:
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])

input = Input(shape=(224,224,3))
x = Rescaling(1./255)(input)
x = data_augmentation(x)
x = model_vgg(x)
x = Rescaling(1./255)(x)
x = Flatten()(x)
output = Dense(1,activation='sigmoid')(x)
model = tf.keras.models.Model(inputs= input,outputs=output)



In [ ]:
import os
import cv2
import tensorflow as tf

def remove_undecodable_images(directory, method="tf"):
    """
    Remove undecodable or empty images from a directory.

    Parameters:
        directory (str): Folder path where images are stored.
        method (str): "cv" for OpenCV, "tf" for TensorFlow
    """
    print(f"Checking directory for undecodable images using {method.upper()}: {directory}")

    for root, _, files in os.walk(directory):
        for file in files:
            filepath = os.path.join(root, file)

            if os.path.isdir(filepath):
                continue

            try:
                if method == "cv":   # ---------- OpenCV Method ----------
                    img = cv2.imread(filepath)
                    if img is None:
                        print(f"Removing undecodable image: {filepath}")
                        os.remove(filepath)
                    elif img.size == 0:
                        print(f"Removing empty image: {filepath}")
                        os.remove(filepath)

                elif method == "tf":  # ---------- TensorFlow Method ----------
                    img_bytes = tf.io.read_file(filepath)
                    img = tf.image.decode_image(img_bytes, channels=3)

                    if tf.size(img).numpy() == 0:
                        print(f"Removing empty image: {filepath}")
                        os.remove(filepath)
                else:
                    print(f"Unknown method '{method}'. Use 'cv' or 'tf'.")
                    return

            except Exception as e:
                print(f"Could not process image {filepath}: {e}")
